<a href="https://colab.research.google.com/github/BrianGisemba/MENTAL-HEALTH-TWEETS-CLASSIFICATION/blob/data-augmentation/Back_Translation_for_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Back Translation for Data Augmentation

In [ ]:
df = pd.read_csv('MeantalHealth.csv')

In [ ]:
GEN_BACK_TR = True

GEN_UPSAMPLE = False

GEN_EN_ONLY = False

In [ ]:
# Importing libraries

!pip install 'fsspec>=0.3.3'
!pip install dask
!pip install googletrans==3.1.0a0
import math, os, re, time
!pip install --quiet googletrans
from googletrans import Translator
from sklearn.model_selection import train_test_split, StratifiedKFold
from dask import bag, diagnostics

In [ ]:
def back_translate(sequence, PROB = 1):
    languages = ['en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh-cn', 'hi','sw', 'vi', 'es', 'el']
    
# Instantiate translator
    translator = Translator()
    
# Store original language so we can convert back
    org_lang = translator.detect(sequence).lang
    
# Randomly choose language to translate sequence to  
    random_lang = np.random.choice([lang for lang in languages if lang is not org_lang])
    
    if org_lang in languages:
        #translate to new language and back to original
        translated = translator.translate(sequence, dest = random_lang).text
        #translate back to original language
        translated_back = translator.translate(translated, dest = org_lang).text
    
        #apply with certain probability
        if np.random.uniform(0, 1) <= PROB:
            output_sequence = translated_back
        else:
            output_sequence = sequence
            
    #if detected language not in our list of languages, do nothing
    else:
        output_sequence = sequence
    
    return output_sequence

#check performance
for i in range(5):
    output = back_translate('I genuinely have no idea what the output of this sequence of words will be')
    print(output)

I really don’t know what the output of this sequence of words will be
I really don't know what the output of this string will look like
I really have no idea the outcome of this string of words
I really don't know what will be the result of this sequence of words
I genuinely have no idea what the output of this sequence of words will be


In [ ]:
#applies above define function with Dask
def back_translate_parallel(dataset):
    tweet_bag = bag.from_sequence(df['tweet'].tolist()).map(back_translate)
    
    with diagnostics.ProgressBar():
        tweets = tweet_bag.compute()

    df[['tweet']] = list(tweets)
    
    return dataset

In [ ]:
if GEN_BACK_TR:
#now we apply translation augmentation
    df_aug = df.pipe(back_translate_parallel)
    

[########################################] | 100% Completed |  6min 33.5s


In [ ]:
df_aug.to_csv('MentalHealth_aug.csv', index = False)